In [5]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
JSON_DATA_PATH = "data/reviews.json"
N = 10

In [7]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt", "helpful", "summary"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        helpful = 0 if d["helpful"][1] == 0 else d["helpful"][0]/d["helpful"][1]
        summary = d["summary"]
        rows.append((uid, iid, review, rating, dt, helpful, summary))
        
    return pd.DataFrame(rows, columns=cols)

In [8]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt,helpful,summary
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000,0.666667,Pay to unlock content? I don't think so.
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400,0.000000,Good rally game
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600,0.000000,Wrong key
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400,0.700000,"awesome game, if it did not crash frequently !!"
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600,1.000000,DIRT 3


In [9]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [10]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

In [11]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
#         print(test_dict[uid].keys())
#         print(recs_dict.get(uid, {}))
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [12]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 7) (46353, 7)
Размер после очистки: (185427, 7) (19174, 7)


In [13]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

In [43]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class NonPersRecommender(BasicRecommender):
    def __init__(self, df):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df)
        
    def _prepare_recs(self, df):
        return pd.Series([])
    
    def get_recs(self, uid, top):
        recs = {}
        if uid in uid_to_row:
            u_row_id = uid_to_row[uid]
            u_row = uid_ftr_m[u_row_id]

            # самописный cosine similarity
            u_recs = u_row.dot(ftr_iid_m)

            for arg_id in np.argsort(u_recs.data)[-top:]:
                row_id = u_recs.indices[arg_id]
                score = u_recs.data[arg_id]
                recs[row_to_iid[row_id]] = score
        return recs
    
    def get_batch_recs(self, uids, top):
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class MostReviewedRS(NonPersRecommender):
    def _prepare_recs(self, df):
        # считаем количество отзывов для каждого объекта (pandas сортирует их по убыванию)

        return df.iid.value_counts()
    
class ContentReviewedRS(NonPersRecommender):
    def _prepare_recs(self, df):
        # считаем количество отзывов для каждого объекта (pandas сортирует их по убыванию)
        

        return df.iid.value_counts()

In [44]:
texts = training_df['review']

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
review_tf = tfidf.fit_transform(texts)

In [46]:
review_tf.tocsr()

<185427x197239 sparse matrix of type '<class 'numpy.float64'>'
	with 14094510 stored elements in Compressed Sparse Row format>

In [47]:
from sklearn.preprocessing import normalize
from scipy.sparse import vstack

def _prepare_iid_data(df, review_ftr_m): 
    iid_to_row = {}
    rows = []
    # не самый оптимальный group by       
    for row_id, iid in enumerate(df.iid.unique()):
        iid_to_row[iid] = row_id
        iid_ftr_m = csr_matrix(
            review_ftr_m[np.where(df.iid == iid)[0]].sum(axis=0)
        )
        rows.append(iid_ftr_m)
    iid_ftr_m = normalize(vstack(rows, format='csr'))
    return iid_to_row, iid_ftr_m

In [48]:
iid_to_row, iid_ftr_m = _prepare_iid_data(training_df, review_tf)

In [49]:
def _prepare_uid_data(df, iid_to_row, iid_ftr_m):  
        uid_to_row = {}
        rows = []
        
        # gr_df - кусок df с данными одного пользователя 
        for gr_id, gr_df in df.groupby("uid"):
            uid = gr_df.uid.values[0]
            
            # поиск объектов и пользовательских рейтингов для них
            iid_rows = []
            ratings = []
            for iid, rating in zip(gr_df.iid.values, gr_df.rating.values):
                if iid in iid_to_row:
                    iid_rows.append(iid_to_row[iid])
                    ratings.append(rating)
                  
            # создание профиля пользователя
            if iid_rows:
                ratings = np.array(ratings).reshape(-1, 1)
                uid_ftr_m = csr_matrix(
                    iid_ftr_m[iid_rows].multiply(ratings).sum(axis=0)
                ) 
                uid_to_row[uid] = len(uid_to_row)
                rows.append(uid_ftr_m)
            
        uid_ftr_m = normalize(vstack(rows, format='csr'))
        return uid_to_row, uid_ftr_m

In [50]:
uid_to_row, uid_ftr_m = _prepare_uid_data(training_df, iid_to_row, iid_ftr_m)

In [51]:
ftr_iid_m = iid_ftr_m.T.tocsr()
row_to_iid = {row_id: iid for iid, row_id in iid_to_row.items()}

In [52]:
def get_recs(uid, top=N):
    recs = {}
    if uid in uid_to_row:
        u_row_id = uid_to_row[uid]
        u_row = uid_ftr_m[u_row_id]

        # самописный cosine similarity
        u_recs = u_row.dot(ftr_iid_m)

        for arg_id in np.argsort(u_recs.data)[-top:]:
            row_id = u_recs.indices[arg_id]
            score = u_recs.data[arg_id]
            recs[row_to_iid[row_id]] = score
    return recs

In [53]:
get_recs(1)

{0: 0.64352899716488576,
 2223: 0.64123706084027632,
 3865: 0.58816280237315743,
 5165: 0.56863420441493806,
 5471: 0.56529268333031935,
 5542: 0.56339220442871918,
 6268: 0.56994552946415011,
 8386: 0.58251005355039065,
 8387: 0.57506475516865396,
 8388: 0.72287952162755142}

In [54]:
test_dict = get_test_dict(test_df)

In [55]:
rs_most_reviewed = ContentReviewedRS(training_df)
recs_dict = rs_most_reviewed.get_batch_recs(test_df.uid, N)
hit_ratio(recs_dict, test_dict )

0.06544387380777696